## Pozyskanie danych
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### Potrzebne pakiety

In [ ]:
import os
import numpy as np
import pandas as pd
from google.cloud import bigquery
import seaborn as sns
import matplotlib.pyplot as plt

## <center>Część 2</center>
### 2.5

In [ ]:
os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"] = "projekt1-415610-f19ebf54e064.json"  # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client()

### Potrzebna funkcja

In [ ]:
def return_df(q):
    query_job = client.query(q)
    query_result = query_job.result()
    return query_result.to_dataframe()

### 2.6

In [ ]:
query = 'select * from `bigquery-public-data.covid19_open_data.covid19_open_data` limit 10'
df = return_df(query)

### Informacje o zbiorze

In [ ]:
df.info()

## <center>Część 3</center>

### 3.1. Sprawdź, ile jest zapisanych wierszy z danymi.

In [ ]:
df_3_1 = return_df('SELECT COUNT(*) FROM `bigquery-public-data.covid19_open_data.covid19_open_data`')
df_3_1

### 3.2. Sprawdź, ile krajów jest uwzględnionych w danych.

In [ ]:
df_3_2 = return_df('SELECT COUNT(DISTINCT country_name) FROM `bigquery-public-data.covid19_open_data.covid19_open_data`')
df_3_2

### 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.

In [ ]:
df_3_3 = return_df('SELECT column_name, data_type FROM `bigquery-public-data`.covid19_open_data.INFORMATION_SCHEMA.COLUMNS WHERE table_name = \'covid19_open_data\''
)
df_3_3

### 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.

In [ ]:
df_3_4 = return_df('SELECT column_name, data_type FROM `bigquery-public-data`.covid19_open_data.INFORMATION_SCHEMA.COLUMNS WHERE table_name = \'covid19_open_data\' AND data_type = \'INT64\' OR data_type = \'FLOAT64\''
)
df_3_4

### 3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.

In [ ]:
queries_3_5 = [
    'SELECT MIN(date) AS min_date, MAX(date) AS max_date FROM `bigquery-public-data.covid19_open_data.covid19_open_data`',
    'SELECT MIN(date) AS min_date_confirmed, MAX(date) AS max_date_confirmed FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE new_confirmed IS NOT NULL',
    'SELECT MIN(date) AS min_date_deceased, MAX(date) AS max_date_deceased FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE new_deceased IS NOT NULL',
    'SELECT MIN(date) AS min_date_vaccinated, MAX(date) AS max_date_vaccinated FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE new_persons_vaccinated IS NOT NULL'
]

df_3_5 = pd.DataFrame()

for query in queries_3_5:
    df_3_5 = pd.concat([df_3_5, return_df(query)], axis=1)
df_3_5

### 3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń.

In [ ]:
queries_3_6 = [
    'SELECT country_name, infant_mortality_rate, adult_male_mortality_rate, adult_female_mortality_rate, ROUND(nurses_per_1000 + physicians_per_1000, 2) AS medical_personnel FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND adult_male_mortality_rate IS NOT NULL AND adult_female_mortality_rate IS NOT NULL GROUP BY country_name, infant_mortality_rate, adult_male_mortality_rate, adult_female_mortality_rate, gdp_usd, nurses_per_1000, physicians_per_1000 ORDER BY gdp_usd DESC LIMIT 10',
    'SELECT subregion1_name, date, testing_policy FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 1 AND country_name=\'Canada\' GROUP BY subregion1_name, date, testing_policy ORDER BY subregion1_name, date',
    'SELECT subregion2_name, date, CAST(new_tested AS int64) AS new_tested, new_confirmed FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 2 AND subregion1_name=\'Łódź\' GROUP BY subregion2_name, date, new_tested, new_confirmed ORDER BY subregion2_name, date',
    'SELECT date, facial_coverings FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE country_name=\'Germany\' AND aggregation_level = 0 GROUP BY date, facial_coverings ORDER BY date',
    'SELECT date, workplace_closing, income_support, CAST(mobility_workplaces AS int64) AS mobility_workplaces FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND country_name=\'United States of America\' GROUP BY date, workplace_closing, income_support, mobility_workplaces ORDER BY date'
]

for query in queries_3_6:
    df_3_6 = return_df(query)
    display(df_3_6.head())

## <center>Część 4</center>

### 4.1 Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

In [ ]:
df_4_1 = return_df('SELECT iso_3166_1_alpha_3 AS CCA3, country_name, population, population_male, population_female, gdp_usd, gdp_per_capita_usd, latitude, longitude, health_expenditure_usd, life_expectancy, population_age_00_09 AS age_bin_0, population_age_10_19 AS age_bin_1, population_age_20_29 AS age_bin_2, population_age_30_39 AS age_bin_3, population_age_40_49 AS age_bin_4, population_age_50_59 AS age_bin_5, population_age_60_69 AS age_bin_6, population_age_70_79 AS age_bin_7, population_age_80_and_older AS age_bin_8 FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND place_id IS NOT NULL AND gdp_usd IS NOT NULL GROUP BY iso_3166_1_alpha_3, country_name, population, population_male, population_female, gdp_usd, gdp_per_capita_usd, latitude, longitude, health_expenditure_usd, life_expectancy, population_age_00_09, population_age_10_19, population_age_20_29, population_age_30_39, population_age_40_49, population_age_50_59, population_age_60_69, population_age_70_79, population_age_80_and_older ORDER BY country_name')

df_4_1.info()

### Uzupełnienie brakujących danych o kraju na podstawie jego <i>sąsiadów</i>

In [ ]:
from haversine import haversine

def save_to_csv(dataframe, filename):
    if 'czesc4' not in os.listdir(os.getcwd()):
        os.mkdir('czesc4')
    os.chdir('czesc4')
    dataframe.to_csv(filename + '.csv') # Zapis do csv
    os.chdir('..')


def fill_based_on_neigh(index, columns, num=10):
    data = df_4_1.loc[index].copy()
    df_4_1['distance'] = df_4_1.apply(lambda x: haversine((data['latitude'], data['longitude']), (x['latitude'], x['longitude'])), axis=1)
    neigh = df_4_1.sort_values(by=['distance'])
    neigh = neigh.iloc[0:num]
    for col in columns:
        if df_4_1[col].dtype == np.float64:
            df_4_1.loc[index, col] = neigh[col].mean()
        else:
            df_4_1.loc[index, col] = int(neigh[col].mean())


def fill_4_1():
    columns = df_4_1.columns
    age_groups = [el for el in columns if 'age_bin_' in el]
    leave = ['CCA3', 'population', 'population_male', 'population_female', 'latitude', 'longitude', 'country_name'] + age_groups
    
    df_4_1.loc[df_4_1['country_name'] == 'Armenia', ['latitude', 'longitude']] = (40.17720, 44.50349)
    
    df_4_1['distance'] = 0.0
    to_float_cols = columns.difference(leave)
    for tfc in to_float_cols:
         df_4_1[tfc] = df_4_1[tfc].astype(np.float64)
    
    df_4_1['population'].fillna(df_4_1['population_male'] + df_4_1['population_female'], inplace=True)
    df_4_1['population_male'].fillna(df_4_1['population'] // 2, inplace = True)
    df_4_1['population_female'].fillna(df_4_1['population'] // 2, inplace = True)
    
    not_equal = df_4_1['population'] != df_4_1['population_male'] + df_4_1['population_female']
    df_4_1['population'].mask(not_equal, df_4_1['population_male'] + df_4_1['population_female'], inplace=True)
    
    for index, row in df_4_1.iterrows():
        if df_4_1.loc[index].isna().sum() > 0:
            fill_based_on_neigh(index=index, columns=df_4_1.columns.difference(leave))
    
    df_4_1.drop('distance', axis=1, inplace=True)
    split = [9, 9, 8, 7, 7, 8, 9, 9]
    for i in range(len(split)):
        df_4_1[age_groups[i]].fillna(df_4_1['population'] // split[i], inplace=True)
    df_4_1['age_bin_8'].fillna(df_4_1['population'] - df_4_1[age_groups].sum(axis=1), inplace=True)


fill_4_1()
save_to_csv(df_4_1, '4_1')

df_4_1.info()

### Informacje o krajach

In [ ]:
df_4_1.head()

### 4.2 Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

In [ ]:
df_4_2 = return_df('SELECT country_name, FORMAT_DATE(\'%B\', date) AS month, SUM(new_confirmed) AS monthly_confirmed, SUM(new_confirmed_male) AS monthly_confirmed_male, SUM(new_confirmed_female) AS monthly_confirmed_female, SUM(new_confirmed_age_0) AS age_confirmed_0, SUM(new_confirmed_age_1) AS age_confirmed_1, SUM(new_confirmed_age_2) AS age_confirmed_2, SUM(new_confirmed_age_3) AS age_confirmed_3, SUM(new_confirmed_age_4) AS age_confirmed_4, SUM(new_confirmed_age_5) AS age_confirmed_5, SUM(new_confirmed_age_6) AS age_confirmed_6, SUM(new_confirmed_age_7) AS age_confirmed_7, SUM(new_confirmed_age_8) + SUM(new_confirmed_age_9) AS age_confirmed_8 FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND place_id IS NOT NULL AND gdp_usd IS NOT NULL GROUP BY country_name, month ORDER BY country_name, month')

df_4_2.info()

### Uzupełnienie danych dotyczących zachorowań na COVID-19

In [ ]:
def fill_values(dataframe):
    columns = dataframe.columns
    
    dataframe[columns[2]].fillna(0, inplace=True)
    
    male_and_female_nan = dataframe[columns[3]].isna() & dataframe[columns[4]].isna()
    rand = np.random.randint(1, 10, size=len(male_and_female_nan))
    dataframe[columns[3]].mask(male_and_female_nan, dataframe[columns[2]] // rand, inplace=True)
    dataframe[columns[4]].mask(male_and_female_nan, dataframe[columns[2]] - dataframe[columns[3]], inplace=True)
    
    only_male_nan = dataframe[columns[3]].isna() & dataframe[columns[4]].notna()
    dataframe[columns[3]].mask(only_male_nan, dataframe[columns[2]] - dataframe[columns[4]], inplace=True)
    
    only_female_nan = dataframe[columns[4]].isna() & dataframe[columns[3]].notna()
    dataframe[columns[4]].mask(only_female_nan, dataframe[columns[2]] - dataframe[columns[3]], inplace=True)
    
    all_is_zero = (dataframe[columns[2]] == 0) & (dataframe[columns[3]] != 0) & (dataframe[columns[4]] != 0)
    dataframe[columns[2]].mask(all_is_zero, dataframe[columns[3]] + dataframe[columns[4]], inplace=True)
    
    # dataframe = most_complete(dataframe)  # można użyć, ale lepiej przewidzieć
    
    ratio = pd.DataFrame({'country_name': df_4_1['country_name']})
    
    age_groups_df = [col for col in columns if 'age_' in col]
    age_groups = [col for col in df_4_1.columns if 'age_' in col]
    
    for i in range(len(age_groups_df)):
        ratio[age_groups_df[i]] = df_4_1[age_groups[i]] / df_4_1['population']
    dataframe = dataframe.merge(ratio, on='country_name', how='left')
    for i in range(len(age_groups_df)):
        dataframe[age_groups_df[i] + '_x'].fillna((dataframe[columns[2]] * dataframe[age_groups_df[i] + '_y']).astype(np.int64), inplace=True)
    dataframe.drop(columns=[age_group + '_y' for age_group in age_groups_df], axis=1, inplace=True)
    dataframe.rename(columns={age_group + '_x': age_group for age_group in age_groups_df}, inplace=True)
    
    return dataframe


def most_complete(dataframe):  # nie interesują nas kraje nie mające żadnych wpisów w jakimkolwiek miesiącu
    nans = dataframe[dataframe.isna().any(axis=1)]['country_name'].value_counts()
    nans = nans[nans.values == 12].index
    return dataframe[~dataframe['country_name'].isin(nans)].reset_index(drop=True)

df_4_2 = fill_values(df_4_2)
save_to_csv(df_4_2, '4_2')

df_4_2.info()

### Miesięczne statystyki zachorowań na COVID-19 dla każdego kraju

In [ ]:
df_4_2.head()

### 4.3 Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

In [ ]:
df_4_3 = return_df('SELECT country_name, FORMAT_DATE(\'%B\', date) AS month, SUM(new_deceased) AS daily_deceased, SUM(new_deceased_male) AS daily_deceased_male, SUM(new_deceased_female) AS daily_deceased_female, SUM(new_deceased_age_0) AS age_deceased_0, SUM(new_deceased_age_1) AS age_deceased_1, SUM(new_deceased_age_2) AS age_deceased_2, SUM(new_deceased_age_3) AS age_deceased_3, SUM(new_deceased_age_4) AS age_deceased_4, SUM(new_deceased_age_5) AS age_deceased_5, SUM(new_deceased_age_6) AS age_deceased_6, SUM(new_deceased_age_7) AS age_deceased_7, SUM(new_deceased_age_8) + SUM(new_deceased_age_9) AS age_deceased_8 FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND place_id IS NOT NULL AND gdp_usd IS NOT NULL GROUP BY country_name, month ORDER BY country_name, month')

df_4_3.info()

### Uzupełnienie danych dotyczących śmiertelności COVID-19

In [ ]:
df_4_3 = fill_values(df_4_3)
save_to_csv(df_4_3, '4_3')

df_4_3.info()

### Miesięczne statystyki śmierci na COVID-19

In [ ]:
df_4_3.head()

### 4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.

In [ ]:
df_4_4 = return_df('SELECT country_name, FORMAT_DATE(\'%B\', date) AS month, SUM(new_persons_vaccinated) AS daily_persons_vaccinated, SUM(new_persons_fully_vaccinated) AS daily_persons_fully_vaccinated FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND place_id IS NOT NULL AND gdp_usd IS NOT NULL GROUP BY country_name, month ORDER BY country_name, month')

df_4_4.info()

### Uzupełnienie danych dotyczących szczepień

In [ ]:
def fill_4_4(dataframe):  # ze względu na wysoki poziom kompletności danych zapełniamy wartościami zero
    return dataframe.fillna(0)


df_4_4 = fill_4_4(df_4_4)
save_to_csv(df_4_4, '4_4')
df_4_4.info()

### Miesięczne statystyki szczepień na COVID-19

In [ ]:
df_4_4.head()

### 4.5. Chcemy przeprowadzić skuteczność polityk zamykania szkół, a zachorowaniami w grupach wiekowych 0-9 i 10-19

In [ ]:
df_4_5 = return_df('SELECT country_name, EXTRACT(MONTH FROM date) AS month, FORMAT_DATE(\'%B\', DATE_TRUNC(date, MONTH)) AS month_name, school_closing, SUM(new_confirmed_age_0) AS preschoolers_confirmed, SUM(new_confirmed_age_1) AS pupils_confirmed FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 AND place_id IS NOT NULL AND gdp_usd IS NOT NULL GROUP BY country_name, month, month_name, school_closing ORDER BY country_name, month, school_closing')

df_4_5.info()

### Czyszczenie danych

In [ ]:
def fill_4_5(dataframe):
    dataframe.drop(columns='month', axis=1, inplace=True)
    dataframe.rename(columns={'month_name': 'month'}, inplace=True)
    dataframe['school_closing'].ffill(inplace=True)
    dataframe['school_closing'].fillna(-1, inplace=True)  # kiedy jeszcze nie było żadnych ograniczeń na początku 2020
    return dataframe.fillna(0)  # albo można proporcjonalnie do populacji w kraju danej grupy wiekowej


df_4_5 = fill_4_5(df_4_5)
save_to_csv(df_4_5, '4_5')
df_4_5.info()

### Miesięczne statystyki zachorowań w populacji do 19 roku życia

In [ ]:
df_4_5.head()

## <center>Część 5</center>
### Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

### Połączenie części 4

In [ ]:
df_5 = pd.merge(df_4_2, df_4_3, on=['country_name', 'month'])
df_5 = pd.merge(df_5, df_4_4, on=['country_name', 'month'])
df_5 = pd.merge(df_5, pd.merge(df_4_1, df_4_5, on=['country_name']), on=['country_name', 'month'])

df_5.info()

In [ ]:
df_5.head()

## <center>Część 6</center>

### 6.1

In [ ]:
df_6_1 = pd.read_csv('czesc6/world_countries.csv')
df_6_1 = pd.merge(df_6_1, df_5, on=['CCA3'])
save_to_csv(df_6_1, '6_1')

df_6_1.info()

In [ ]:
df_6_1.head()

### 6.2

In [ ]:
df_6_2 = pd.read_csv('czesc6/gdp.csv')
df_6_2.rename(columns={'Country Name': 'country_name', 'Country Code': 'CCA3'}, inplace=True)
df_6_2 = pd.merge(df_6_2, df_5, on=['country_name', 'CCA3'])
save_to_csv(df_6_2, '6_2')

df_6_2.info()

In [ ]:
df_6_2.head()

## <center>Część 7</center>
### Znajdź 3 dodatkowe zbiory danych, które uważasz, że rozwijają w ciekawy sposób temat ostatniej pandemii. Powtórz dla nich analogiczne czynności z części 4, a następnie połącz je z danymi z części 6. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.